In [1]:
# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 

True

In [2]:
from llama_index.core import StorageContext, load_index_from_storage
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")


In [2]:
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="C:/Users/johnl/OneDrive/Documents/GitHub/EldenBossBuster/index/data") # location of those json files

# load index
index = load_index_from_storage(storage_context)

In [ ]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(
    model="gpt-4o",
    temperature=0.7,  # creativity
    top_p=0.95,       # sampling diversity
    frequency_penalty=0.2,  # reduce repetitive answers
    num_outputs=1000, # tokens in the output
)

In [ ]:
# agent to refine original prompt
prompt_refine_agent = openai.OpenAI(api_key=openai.api_key)

original_query= "Recommended full set of equipment for playing with Unsheathe to beat Margit"

messages = [
    {"role": "system",
    "content": "You are a prompt refining agent that helps to refine the prompt to another agent. \
        The job of the other agent is to recommend strategy based on player's status.\
        The game is Elden Ring. \
        Please use vocabularies from the Elden Ring to refine the prompt when suitable.\
        Given the below prompt, refine it so it is better to be feed into the recommendation agent that recommend a strategy."
    },
    {
    "role": "user", 
    "content": f"{original_query}"
    }
]
response = prompt_refine_agent.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    max_tokens=1000,
    n=1,
    top_p=0.95,
    frequency_penalty=0.2,
    temperature=0.7
)

query_to_recommendation_agent = response.choices[0].message.content
print(query_to_recommendation_agent)

Suggest an optimal equipment set for a player utilizing the Unsheathe skill to defeat Margit, the Fell Omen.


In [20]:
query_enginge = index.as_query_engine(response_mode="refine", similarity_top_k=5, verbose = True, llm = llm)

In [21]:
response = query_enginge.query(f"{query_to_recommendation_agent}")

In [22]:
print(response)

To effectively defeat Margit using the Unsheathe skill, equip the Uchigatana, which pairs well with this combat style. Use the Keen Asymmeters +2 for optimal damage, and consider Margit's Shackle to temporarily immobilize him for free hits. Summon NPCs like Sorcerer Rogier or use Spirit Ashes such as Lone Wolf Ashes for distraction. Equip the Turtle Talisman to enhance stamina recovery, and ensure your armor provides decent physical damage negation, ideally around 100%, while maintaining agility for swift Unsheathe attacks. Prioritize high dexterity to maximize the damage output of the Uchigatana.
